## 카카오 지오코딩 완성

In [2]:
import pandas as pd
import re
import requests, json

# 위경도 찾을 파일 불러오기
file_path = './2. 충전소/경기도시별충전소_1.csv'
ggi_1 = pd.read_csv(file_path)

# 주소 전처리
pattern = r'\([^)]*\)'
text = []

for i in range(0, len(ggi_1['주소']),1):
    text.append(re.sub(pattern=pattern, repl='', string= ggi_1['주소'][i]))

text
ggi_1['주소'] = text

# 카카오API를 사용하여 주소 -> 좌표 변환
def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
  # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
  headers = {"Authorization": "KakaoAK 22e701c1d44c5fe8b6441a9983e059f6"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"lat": str(address['y']), "lng": str(address['x'])}
  address_name = address['address_name']

  return crd

# 파일에서 주소를 위경도로 변환해서 crd에 넣기
crd = []

for i in range(0, len(ggi_1['주소']), 1):
    try:
        crd.append(get_location(ggi_1['주소'][i]))
    except:
        crd.append({'lat':0,'lng':0})

# crd를 데이터프레임화 하여 원래 파일에 붙이기
gg = pd.DataFrame(crd)
ggi_1 = pd.concat([ggi_1,gg], axis=1)
ggi_1

,Unnamed: 0,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한,주소2,lat,lng
0,65,환경부(한국자동차환경협회),관악역 3환승 공영주차장,1,DC차데모+AC3상+DC콤보,경기도,안양시,경기도 안양시 만안구 석수1동 110-26 입구 맨끝 왼쪽,제한없음,안구 석수1동 110-26 입구 맨끝 왼쪽,37.4184843494964,126.909735703032
1,72,환경부(한국자동차환경협회),구리남양주 교육지원청,1,DC콤보,경기도,남양주시,경기도 남양주시 경춘로 520,제한없음,경춘로 520,37.6108432181515,127.169814910409
2,73,환경부(한국자동차환경협회),구리남양주 교육지원청,2,DC콤보,경기도,남양주시,경기도 남양주시 경춘로 520,제한없음,경춘로 520,37.6108432181515,127.169814910409
3,74,환경부(한국자동차환경협회),구리남양주 교육지원청,3,DC콤보,경기도,남양주시,경기도 남양주시 경춘로 520,제한없음,경춘로 520,37.6108432181515,127.169814910409
4,81,환경부(한국자동차환경협회),구성동주민센터,1,DC콤보,경기도,용인시,경기도 용인시 기흥구 구성로77번길 17,제한없음,흥구 구성로77번길 17,37.2967640172842,127.118981468612
...,...,...,...,...,...,...,...,...,...,...,...,...
7079,11337,캐스트프로,성남_우림라이온스밸리1차,7,AC완속,경기도,성남시,"경기도 성남시 중원구 둔촌대로457번길 272층, 3층 주차장",제한없음,"원구 둔촌대로457번길 272층, 3층 주차장(311-3 성남우림라이온스밸리 2층,...",0,0
7080,11338,캐스트프로,성남_우림라이온스밸리1차,8,AC완속,경기도,성남시,"경기도 성남시 중원구 둔촌대로457번길 272층, 3층 주차장",제한없음,"원구 둔촌대로457번길 272층, 3층 주차장(311-3 성남우림라이온스밸리 2층,...",0,0
7081,11339,캐스트프로,캐스트프로1채널테스트,1,AC완속,경기도,성남시,경기도 성남시 중원구 둔촌대로457번길 271312호,제한없음,원구 둔촌대로457번길 271312호(311-3 성남우림라이온스밸리 1312호),0,0
7082,11340,캐스트프로,캐스트프로테스,1,DC차데모+AC3상,경기도,성남시,경기도 성남시 중원구 둔촌대로457번길 2713F,제한없음,원구 둔촌대로457번길 2713F(311-3 성남우림라이온스밸리),0,0
